# Import library

In [1]:
import os
import os, sys
import glob
import re
import datetime
from datetime import datetime, timedelta 
import math, copy, time
from itertools import cycle
from collections import defaultdict

from parse import parse
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

# import torchvision
# import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn import svm


from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn

%load_ext autoreload
%autoreload 2

/Users/mckim/miniconda3/envs/py38/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
pd.set_option("display.max_column", 100)

In [6]:
# Util functions
def size_and_ratio(df, col, dropna=True):
    if pd.__version__ > '1.1':
        sr = df.groupby(col, dropna=dropna).size().sort_values(ascending=False)
    elif dropna:
        sr = df.groupby(col).size().sort_values(ascending=False)
    else:  # dropna == False
        sr = df.fillna('nan').groupby(col).size().sort_values(ascending=False)
    sr_ratio = sr.copy() / sum(sr)
    print("Sum :", sum(sr), sr.shape)
    sr.name = 'size'
    sr_ratio.name = 'ratio'
    return pd.concat([sr, sr_ratio], axis=1)

# Data load

In [7]:
df_train = pd.read_csv("data/df_data_v3_1/df_train_v3_1.csv")
df_val = pd.read_csv("data/df_data_v3_1/df_valid_v3_1.csv")
df_test = pd.read_csv("data/df_data_v3_1/df_test_v3_1.csv")
print(df_train.shape, df_val.shape, df_test.shape)
df_train.head()

(14093, 24) (4647, 24) (4586, 24)


,baby_id,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday,pcr_result
0,21099,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0.173311,0.007956,0.012987,1,0.0,0.18,3,0
1,21151,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.179009,0.008614,0.013084,1,0.0,0.17,4,0
2,21024,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.179009,0.008614,0.013084,1,1.0,0.29,4,0
3,21176,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,0.0,0.40,5,0
4,4467,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,1.0,0.34,5,0


In [8]:
import random
print("Train:", df_train.shape, df_train['baby_id'].nunique())
print("Validaiton:", df_val.shape, df_val['baby_id'].nunique())

Train: (14093, 24) 9641
Validaiton: (4647, 24) 3213


## Remove baby_id and results

In [9]:
exclude_cols = ['baby_id', 'pcr_result']
cols = [x for x in df_train.columns if x not in exclude_cols]

df_train_gt = df_train[['pcr_result']]
df_train_input = df_train[cols]
df_val_gt = df_val[['pcr_result']]
df_val_input = df_val[cols]
df_test_gt = df_test[['pcr_result']]
df_test_input = df_test[cols]


# Make category values

In [12]:
from model.attention_intensity_after_encoder import MultiplyIntensityModel
from model.utils import get_dict_category_from_dataset
from model.utils import get_category_key, create_cat_and_intensity_from_df

In [13]:
df_trainval = pd.concat([df_train, df_val], axis=0)
print(df_trainval.shape)

dict_category, df_trainval_typed = get_dict_category_from_dataset(df_trainval[cols])
_, df_train_typed = get_dict_category_from_dataset(df_train[cols])
_, df_val_typed = get_dict_category_from_dataset(df_val[cols])
_, df_test_typed = get_dict_category_from_dataset(df_test[cols])
dict_category

(18740, 24)
cough category [1 0] ...
fever category [0 1] ...
sore_throat category [0 1] ...
shortness_of_breath category [0 1] ...
head_ache category [1 0] ...
runny_nose category [0 1] ...
muscle_pain category [0 1] ...
chills category [0 1] ...
loss_of_taste category [0 1] ...
loss_of_smell category [0 1] ...
sputum category [1 0] ...
chest_pain category [0 1] ...
indication_other category [0 1] ...
indication_abroad category [0 1] ...
indication_contact category [1 0] ...
global_confirmed_ratio float [0.1733114  0.17900909 0.13149977 0.14631498 0.15655098 0.17425644
 0.17103502 0.16990724 0.12749546 0.14885249]
confirmed_ratio float [0.00795568 0.00861396 0.00825183 0.00797017 0.01154644 0.01142734
 0.0113018  0.01122937 0.01076423 0.00895517]
sigungu_confirmed_ratio float [0.01298674 0.01308374 0.         0.01324623 0.01342084 0.0135712
 0.01798011 0.01397984 0.01779546 0.0141302 ]
mask category [1 0] ...
gender category [0. 1.] ...
age_ratio float [0.18 0.17 0.29 0.4  0.34 0.41 0

{'cough_1.0': 0,
 'cough_0.0': 1,
 'fever_0.0': 2,
 'fever_1.0': 3,
 'sore_throat_0.0': 4,
 'sore_throat_1.0': 5,
 'shortness_of_breath_0.0': 6,
 'shortness_of_breath_1.0': 7,
 'head_ache_1.0': 8,
 'head_ache_0.0': 9,
 'runny_nose_0.0': 10,
 'runny_nose_1.0': 11,
 'muscle_pain_0.0': 12,
 'muscle_pain_1.0': 13,
 'chills_0.0': 14,
 'chills_1.0': 15,
 'loss_of_taste_0.0': 16,
 'loss_of_taste_1.0': 17,
 'loss_of_smell_0.0': 18,
 'loss_of_smell_1.0': 19,
 'sputum_1.0': 20,
 'sputum_0.0': 21,
 'chest_pain_0.0': 22,
 'chest_pain_1.0': 23,
 'indication_other_0.0': 24,
 'indication_other_1.0': 25,
 'indication_abroad_0.0': 26,
 'indication_abroad_1.0': 27,
 'indication_contact_1.0': 28,
 'indication_contact_0.0': 29,
 'global_confirmed_ratio': 30,
 'confirmed_ratio': 31,
 'sigungu_confirmed_ratio': 32,
 'mask_1.0': 33,
 'mask_0.0': 34,
 'gender_0.0': 35,
 'gender_1.0': 36,
 'age_ratio': 37,
 'weekday_3.0': 38,
 'weekday_4.0': 39,
 'weekday_5.0': 40,
 'weekday_0.0': 41,
 'weekday_1.0': 42,
 'wee

### category 값, intensity 값 생성

### train 데이터 먼저

In [14]:
df_train_category, df_train_intensity = create_cat_and_intensity_from_df(df_train_typed, dict_category)

display(df_train_category.sample(10))
display(df_train_intensity.sample(10))

,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday
1296,0.0,3.0,5.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,36.0,37.0,39.0
9665,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,34.0,35.0,37.0,41.0
8860,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,24.0,26.0,28.0,30.0,31.0,32.0,33.0,36.0,37.0,43.0
2003,0.0,3.0,5.0,6.0,8.0,11.0,12.0,15.0,16.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0,32.0,33.0,36.0,37.0,42.0
4839,0.0,2.0,5.0,6.0,8.0,11.0,12.0,14.0,16.0,18.0,20.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,36.0,37.0,38.0
4734,0.0,2.0,5.0,6.0,8.0,11.0,12.0,14.0,16.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0,32.0,33.0,36.0,37.0,38.0
12384,0.0,2.0,5.0,6.0,8.0,11.0,13.0,15.0,16.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0,32.0,34.0,36.0,37.0,42.0
11151,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,34.0,36.0,37.0,42.0
13515,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,36.0,37.0,42.0
710,0.0,2.0,5.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0,32.0,33.0,36.0,37.0,41.0


,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday
5751,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.364473,0.569315,0.206732,1.0,1.0,0.41,1.0
13801,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.161099,0.055865,0.000000,1.0,1.0,0.30,1.0
14014,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.104188,0.070357,0.000000,1.0,1.0,0.25,1.0
12528,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.207162,0.241131,0.000000,1.0,1.0,0.28,1.0
10533,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.230106,0.179103,0.000000,1.0,1.0,0.31,1.0
9611,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.232620,0.339080,0.000000,1.0,1.0,0.47,1.0
3158,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.440369,0.563603,0.000000,1.0,1.0,0.24,1.0
2991,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.440369,0.563603,0.000000,1.0,1.0,0.30,1.0
13230,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.164664,0.144094,0.506402,1.0,1.0,0.41,1.0
13825,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.142174,0.046036,0.000000,1.0,1.0,0.53,1.0


예시로 sore_throat id값 확인해봄

In [15]:
size_and_ratio(df_train_category, 'sore_throat')

Sum : 14093 (2,)


,size,ratio
sore_throat,,
5.0,7723,0.548003
4.0,6370,0.451997


### df_val에서 df_cat값과 df_intensity 값 생성

In [16]:
df_val_category, df_val_intensity = create_cat_and_intensity_from_df(df_val_typed, dict_category)


### df_test에서도 마찬가지로 생성

In [17]:
df_test_category, df_test_intensity = create_cat_and_intensity_from_df(df_test_typed, dict_category)


In [18]:
train_np_cat = df_train_category.values
train_np_ins = df_train_intensity.values
train_np_gt = df_train_gt[['pcr_result']].values

val_np_cat = df_val_category.values
val_np_ins = df_val_intensity.values
val_np_gt = df_val_gt[['pcr_result']].values

test_np_cat = df_test_category.values
test_np_ins = df_test_intensity.values
test_np_gt = df_test_gt[['pcr_result']].values

train_np_gt, val_np_gt, test_np_gt

(array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [0]]),
 array([[0],
        [0],
        [0],
        ...,
        [0],
        [1],
        [0]]),
 array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [1]]))

In [19]:
train_input = [train_np_cat.copy(), train_np_ins.copy()]
train_gt = train_np_gt.copy()

valid_input = [val_np_cat.copy(), val_np_ins.copy()]
valid_gt = val_np_gt.copy()

test_input = [test_np_cat.copy(), test_np_ins.copy()]
test_gt = test_np_gt.copy()


In [20]:
print(train_input[0].shape, valid_input[0].shape, test_input[0].shape)
print(train_gt.shape, valid_gt.shape, test_gt.shape)

(14093, 22) (4647, 22) (4586, 22)
(14093, 1) (4647, 1) (4586, 1)


In [21]:
print("# of train data positive: ", train_gt.sum())
print("# of train data negative: ", train_gt.shape[0] - train_gt.sum())
print("ratio of P/N: ", train_gt.sum()/(train_gt.shape[0] - train_gt.sum()))
print("")

print("# of valid data positive: ", valid_gt.sum())
print("# of valid data negative: ", valid_gt.shape[0] - valid_gt.sum())
print("ratio of P/N: ", valid_gt.sum()/(valid_gt.shape[0] - valid_gt.sum()))
print("")

print("# of test data positive: ", test_gt.sum())
print("# of test data negative: ", test_gt.shape[0] - test_gt.sum())
print("ratio of P/N: ", test_gt.sum()/(test_gt.shape[0] - test_gt.sum()))

# of train data positive:  10063
# of train data negative:  4030
ratio of P/N:  2.4970223325062033

# of valid data positive:  3415
# of valid data negative:  1232
ratio of P/N:  2.7719155844155843

# of test data positive:  3332
# of test data negative:  1254
ratio of P/N:  2.657097288676236


### Check dict_category

In [22]:
dict_category

{'cough_1.0': 0,
 'cough_0.0': 1,
 'fever_0.0': 2,
 'fever_1.0': 3,
 'sore_throat_0.0': 4,
 'sore_throat_1.0': 5,
 'shortness_of_breath_0.0': 6,
 'shortness_of_breath_1.0': 7,
 'head_ache_1.0': 8,
 'head_ache_0.0': 9,
 'runny_nose_0.0': 10,
 'runny_nose_1.0': 11,
 'muscle_pain_0.0': 12,
 'muscle_pain_1.0': 13,
 'chills_0.0': 14,
 'chills_1.0': 15,
 'loss_of_taste_0.0': 16,
 'loss_of_taste_1.0': 17,
 'loss_of_smell_0.0': 18,
 'loss_of_smell_1.0': 19,
 'sputum_1.0': 20,
 'sputum_0.0': 21,
 'chest_pain_0.0': 22,
 'chest_pain_1.0': 23,
 'indication_other_0.0': 24,
 'indication_other_1.0': 25,
 'indication_abroad_0.0': 26,
 'indication_abroad_1.0': 27,
 'indication_contact_1.0': 28,
 'indication_contact_0.0': 29,
 'global_confirmed_ratio': 30,
 'confirmed_ratio': 31,
 'sigungu_confirmed_ratio': 32,
 'mask_1.0': 33,
 'mask_0.0': 34,
 'gender_0.0': 35,
 'gender_1.0': 36,
 'age_ratio': 37,
 'weekday_3.0': 38,
 'weekday_4.0': 39,
 'weekday_5.0': 40,
 'weekday_0.0': 41,
 'weekday_1.0': 42,
 'wee

## Let's Train imported class (simple attention module without inesity encoding)

In [24]:
from model.attention_intensity_after_encoder import MultiplyIntensityModel as MultiIntenAfterEmbeding
from model.base_dataloader import IsraelDataLoader, IsraelDatasetWithIntensity
from model.utils import run_validation

In [25]:
train_input

[array([[ 0.,  2.,  4., ..., 35., 37., 38.],
        [ 1.,  2.,  5., ..., 35., 37., 39.],
        [ 1.,  2.,  5., ..., 36., 37., 39.],
        ...,
        [ 0.,  2.,  5., ..., 36., 37., 41.],
        [ 1.,  2.,  4., ..., 36., 37., 41.],
        [ 1.,  2.,  4., ..., 36., 37., 41.]]),
 array([[1.  , 1.  , 1.  , ..., 1.  , 0.18, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.17, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.29, 1.  ],
        ...,
        [1.  , 1.  , 1.  , ..., 1.  , 0.31, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.29, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.46, 1.  ]])]

In [26]:
print("cat value:", train_input[0].shape)
print("intensity value:", train_input[1].shape)
print("Ground truth value:", train_gt.shape)

cat value: (14093, 22)
intensity value: (14093, 22)
Ground truth value: (14093, 1)


# Dataset, data_loader

In [27]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [28]:
# creat dataset

train_dataset = IsraelDatasetWithIntensity(
    train_np_cat,
    train_np_ins,
    train_np_gt,
    dict_category
)

val_dataset = IsraelDatasetWithIntensity(
    val_np_cat,
    val_np_ins,
    val_np_gt,
    dict_category
)

test_dataset = IsraelDatasetWithIntensity(
    test_np_cat, 
    test_np_ins,
    test_np_gt,
    dict_category
)

In [29]:
def save_model(config_name, model, epoch):
    save_path = f"./train_data/{model.__class__.__name__}_{config_name}_{epoch:04d}.pth"
    print("save_path", save_path)
    torch.save(model.state_dict(), save_path)
    print("Success to save to : ", save_path)
    return model

## MultiIntenAfterEmbeding Model with dropout

In [31]:

len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)
d_model = 64
n_head = 8
lr = 1e-5

model = MultiIntenAfterEmbeding(
    dict_category, d_model=d_model, n_head=n_head, len_seq=train_dataset.dataset_np_cat.shape[1],
    dropout=0.1
)

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss()

optimizer = optim.RAdam(model.parameters(), lr=lr)
batch_size = 32
epochs = 300

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

config_name = f"dmodel{d_model}_nhead{n_head}_lr{lr}"
train_log = dict()
val_log = dict()
loss_func = nn.BCELoss()
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}

model.train()
for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = MultiIntenAfterEmbeding.train_on_epoch(epoch, train_loader, model, loss_func, optimizer)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = MultiIntenAfterEmbeding.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result

    logger.info(f"epoch {epoch} val auc: {val_result['auc']}")

    if not val_log:
        best_model = save_model(config_name, model.copy(), epoch)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = save_model( config_name, model.copy(), epoch)
    

len_seq: 22
d_model: 64 n_head: 8 len_seq: 22


INFO:model.base_model:Epoch Step: 0 Train Loss: 0.6470720171928406 elapsed: 1.981964111328125
INFO:model.base_model:cur_epoch:0, validation elapsed_time:0.19272089004516602
INFO:__main__:epoch 0 val auc: 0.49170319065999885


save_path ./train_data/MultiplyIntensityModel_dmodel64_nhead8_lr1e-05_0000.pth
Success to save to :  ./train_data/MultiplyIntensityModel_dmodel64_nhead8_lr1e-05_0000.pth


INFO:model.base_model:Epoch Step: 1 Train Loss: 0.6163455843925476 elapsed: 2.1821510791778564
INFO:model.base_model:cur_epoch:1, validation elapsed_time:0.20082712173461914
INFO:__main__:epoch 1 val auc: 0.7046657222718716


save_path ./train_data/MultiplyIntensityModel_dmodel64_nhead8_lr1e-05_0001.pth
Success to save to :  ./train_data/MultiplyIntensityModel_dmodel64_nhead8_lr1e-05_0001.pth


...

INFO:model.base_model:Epoch Step: 130 Train Loss: 0.4484095275402069 elapsed: 1075.7105672359467
INFO:model.base_model:cur_epoch:130, validation elapsed_time:0.20357704162597656
INFO:__main__:epoch 130 val auc: 0.8340739147382632
INFO:model.base_model:Epoch Step: 131 Train Loss: 0.4507479667663574 elapsed: 3.497046947479248
INFO:model.base_model:cur_epoch:131, validation elapsed_time:0.20012497901916504
INFO:__main__:epoch 131 val auc: 0.8337283232872544
INFO:model.base_model:Epoch Step: 132 Train Loss: 0.43924665451049805 elapsed: 989.8362030982971
INFO:model.base_model:cur_epoch:132, validation elapsed_time:0.20619416236877441
INFO:__main__:epoch 132 val auc: 0.8359429370044305
INFO:model.base_model:Epoch Step: 133 Train Loss: 0.43763306736946106 elapsed: 3.455157995223999
INFO:model.base_model:cur_epoch:133, validation elapsed_time:1065.9542350769043
INFO:__main__:epoch 133 val auc: 0.8367428599950563
INFO:model.base_model:Epoch Step: 134 Train Loss: 0.4466904401779175 elapsed: 3.50

INFO:__main__:epoch 165 val auc: 0.8295831273411801
INFO:model.base_model:Epoch Step: 166 Train Loss: 0.4503549337387085 elapsed: 3.5644888877868652
INFO:model.base_model:cur_epoch:166, validation elapsed_time:0.21562600135803223
INFO:__main__:epoch 166 val auc: 0.8344280627864082
INFO:model.base_model:Epoch Step: 167 Train Loss: 0.44756653904914856 elapsed: 3.6343066692352295
INFO:model.base_model:cur_epoch:167, validation elapsed_time:0.2043137550354004
INFO:__main__:epoch 167 val auc: 0.8346614677416289
INFO:model.base_model:Epoch Step: 168 Train Loss: 0.4318866729736328 elapsed: 3.612353801727295
INFO:model.base_model:cur_epoch:168, validation elapsed_time:0.20515680313110352
INFO:__main__:epoch 168 val auc: 0.8317156214941721
INFO:model.base_model:Epoch Step: 169 Train Loss: 0.44235384464263916 elapsed: 3.529707908630371
INFO:model.base_model:cur_epoch:169, validation elapsed_time:0.20418810844421387
INFO:__main__:epoch 169 val auc: 0.8315822811887965
INFO:model.base_model:Epoch S

INFO:__main__:epoch 201 val auc: 0.831713482344888
INFO:model.base_model:Epoch Step: 202 Train Loss: 0.44344303011894226 elapsed: 3.5219650268554688
INFO:model.base_model:cur_epoch:202, validation elapsed_time:0.20697307586669922
INFO:__main__:epoch 202 val auc: 0.8314022361240516
INFO:model.base_model:Epoch Step: 203 Train Loss: 0.44544997811317444 elapsed: 3.5034351348876953
INFO:model.base_model:cur_epoch:203, validation elapsed_time:0.20299887657165527
INFO:__main__:epoch 203 val auc: 0.8295675590880569
INFO:model.base_model:Epoch Step: 204 Train Loss: 0.442333459854126 elapsed: 3.743587017059326
INFO:model.base_model:cur_epoch:204, validation elapsed_time:0.2021040916442871
INFO:__main__:epoch 204 val auc: 0.8282966667300489
INFO:model.base_model:Epoch Step: 205 Train Loss: 0.45266327261924744 elapsed: 3.5531961917877197
INFO:model.base_model:cur_epoch:205, validation elapsed_time:0.20588111877441406
INFO:__main__:epoch 205 val auc: 0.8322384058108802
INFO:model.base_model:Epoch S

INFO:__main__:epoch 237 val auc: 0.8327261318476544
INFO:model.base_model:Epoch Step: 238 Train Loss: 0.4365302622318268 elapsed: 3.6050000190734863
INFO:model.base_model:cur_epoch:238, validation elapsed_time:0.21931910514831543
INFO:__main__:epoch 238 val auc: 0.8296941254206993
INFO:model.base_model:Epoch Step: 239 Train Loss: 0.4402286112308502 elapsed: 3.7307448387145996
INFO:model.base_model:cur_epoch:239, validation elapsed_time:0.20314979553222656
INFO:__main__:epoch 239 val auc: 0.832496054457987
INFO:model.base_model:Epoch Step: 240 Train Loss: 0.4503314197063446 elapsed: 3.4391798973083496
INFO:model.base_model:cur_epoch:240, validation elapsed_time:0.19852304458618164
INFO:__main__:epoch 240 val auc: 0.8349300498184099
INFO:model.base_model:Epoch Step: 241 Train Loss: 0.4508661925792694 elapsed: 3.4022819995880127
INFO:model.base_model:cur_epoch:241, validation elapsed_time:0.20721006393432617
INFO:__main__:epoch 241 val auc: 0.8325277851723678
INFO:model.base_model:Epoch S

INFO:__main__:epoch 273 val auc: 0.8334381120343785
INFO:model.base_model:Epoch Step: 274 Train Loss: 0.443307101726532 elapsed: 3.6080949306488037
INFO:model.base_model:cur_epoch:274, validation elapsed_time:0.20587682723999023
INFO:__main__:epoch 274 val auc: 0.8325057994713925
INFO:model.base_model:Epoch Step: 275 Train Loss: 0.44468939304351807 elapsed: 3.530225992202759
INFO:model.base_model:cur_epoch:275, validation elapsed_time:0.20328688621520996
INFO:__main__:epoch 275 val auc: 0.8330729117149321
INFO:model.base_model:Epoch Step: 276 Train Loss: 0.44732362031936646 elapsed: 3.6768462657928467
INFO:model.base_model:cur_epoch:276, validation elapsed_time:0.22020816802978516
INFO:__main__:epoch 276 val auc: 0.8283541860774658
INFO:model.base_model:Epoch Step: 277 Train Loss: 0.45117267966270447 elapsed: 3.623532772064209
INFO:model.base_model:cur_epoch:277, validation elapsed_time:0.22741389274597168
INFO:__main__:epoch 277 val auc: 0.8300918883459147
INFO:model.base_model:Epoch 

In [32]:
## Check the best model's validation performance
display(val_log)
print("Best validation performance:", max(val_log.values(), key=lambda x : x['auc']))

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = best_model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

{'epoch_val_0': {'loss': tensor(0.5832), 'auc': 0.49170319065999885},
 'epoch_val_1': {'loss': tensor(0.6163), 'auc': 0.7046657222718716},
 'epoch_val_2': {'loss': tensor(0.5528), 'auc': 0.8101829448004412},
 'epoch_val_3': {'loss': tensor(0.5395), 'auc': 0.818311355555133},
 'epoch_val_4': {'loss': tensor(0.4693), 'auc': 0.8180320777319314},
 'epoch_val_5': {'loss': tensor(0.4590), 'auc': 0.8180219761936454},
 'epoch_val_6': {'loss': tensor(0.4921), 'auc': 0.8189388393451351},
 'epoch_val_7': {'loss': tensor(0.4776), 'auc': 0.8240227653020478},
 'epoch_val_8': {'loss': tensor(0.4448), 'auc': 0.8249365385712384},
 'epoch_val_9': {'loss': tensor(0.4558), 'auc': 0.8135950067502042},
 'epoch_val_10': {'loss': tensor(0.4554), 'auc': 0.8073993173736951},
 'epoch_val_11': {'loss': tensor(0.4420), 'auc': 0.825150691182902},
 'epoch_val_12': {'loss': tensor(0.4435), 'auc': 0.8286610351581069},
 'epoch_val_13': {'loss': tensor(0.4502), 'auc': 0.8266463130573671},
 'epoch_val_14': {'loss': tenso

INFO:model.base_model:cur_epoch:299, validation elapsed_time:0.1710519790649414


Best validation performance: {'loss': tensor(0.4364), 'auc': 0.8388633511437318}


{'loss': tensor(0.4372), 'auc': 0.8372966411444961}

### Add scaled intensity model
also smooth_label

In [98]:
from model.attention_model import CategoricalAttentionAddScaledIntensityModel
from model.label_smoothing import LabelSmoothingBCELoss

len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)
d_model = 64
n_head = 8
lr = 1e-5
dropout = 0.1
smoothing = 0.1

model = CategoricalAttentionAddScaledIntensityModel(
    dict_category, d_model=d_model, n_head=n_head, len_seq=train_dataset.dataset_np_cat.shape[1],
    dropout=dropout
)

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = LabelSmoothingBCELoss(smoothing=smoothing)

optimizer = optim.RAdam(model.parameters(), lr=lr)
batch_size = 32
epochs = 300

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

config_name = f"dmodel{d_model}_nhead{n_head}_lr{lr}_dropout{dropout}_labelsoomthing{smoothing}"
train_log = dict()
val_log = dict()
loss_func = LabelSmoothingBCELoss(smoothing=0.1)
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}

model.train()
for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalAttentionAddScaledIntensityModel.train_on_epoch(epoch, train_loader, model, loss_func, optimizer)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = CategoricalAttentionAddScaledIntensityModel.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result

    logger.info(f"epoch {epoch} val auc: {val_result['auc']}")


    if not val_log:
        best_model = save_model(config_name, model.copy(), epoch)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = save_model( config_name, model.copy(), epoch)


len_seq: 22
d_model: 64 n_head: 8 len_seq: 22


INFO:model.base_model:Epoch Step: 0 Train Loss: 0.6736512780189514 elapsed: 2.081171989440918
INFO:model.base_model:cur_epoch:0, validation elapsed_time:0.2511439323425293
INFO:__main__:epoch 0 val auc: 0.43158442033808064


save_path ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0000.pth
Success to save to :  ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0000.pth


INFO:model.base_model:Epoch Step: 1 Train Loss: 0.6752747297286987 elapsed: 2.3667352199554443
INFO:model.base_model:cur_epoch:1, validation elapsed_time:0.2377490997314453
INFO:__main__:epoch 1 val auc: 0.5952863845524899


save_path ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0001.pth
Success to save to :  ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0001.pth


...

INFO:model.base_model:cur_epoch:49, validation elapsed_time:0.2523996829986572
INFO:__main__:epoch 49 val auc: 0.7857322307999467
INFO:model.base_model:Epoch Step: 50 Train Loss: 0.569364607334137 elapsed: 2.5307838916778564
INFO:model.base_model:cur_epoch:50, validation elapsed_time:0.2591519355773926
INFO:__main__:epoch 50 val auc: 0.7965938801315815
INFO:model.base_model:Epoch Step: 51 Train Loss: 0.5729973316192627 elapsed: 2.547887086868286
INFO:model.base_model:cur_epoch:51, validation elapsed_time:0.3017539978027344
INFO:__main__:epoch 51 val auc: 0.7842891369245688
INFO:model.base_model:Epoch Step: 52 Train Loss: 0.5740832090377808 elapsed: 2.5038599967956543
INFO:model.base_model:cur_epoch:52, validation elapsed_time:0.277371883392334
INFO:__main__:epoch 52 val auc: 0.7798015582514118
INFO:model.base_model:Epoch Step: 53 Train Loss: 0.5782356858253479 elapsed: 2.6531550884246826
INFO:model.base_model:cur_epoch:53, validation elapsed_time:0.2586250305175781
INFO:__main__:epoch 

save_path ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0054.pth
Success to save to :  ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0054.pth


INFO:model.base_model:Epoch Step: 55 Train Loss: 0.5783951282501221 elapsed: 2.5143258571624756
INFO:model.base_model:cur_epoch:55, validation elapsed_time:0.24967193603515625
INFO:__main__:epoch 55 val auc: 0.7947980643075812
INFO:model.base_model:Epoch Step: 56 Train Loss: 0.5766581892967224 elapsed: 2.615615129470825
INFO:model.base_model:cur_epoch:56, validation elapsed_time:0.24566984176635742
INFO:__main__:epoch 56 val auc: 0.7976022513357799
INFO:model.base_model:Epoch Step: 57 Train Loss: 0.5773080587387085 elapsed: 2.509709119796753
INFO:model.base_model:cur_epoch:57, validation elapsed_time:0.24942398071289062
INFO:__main__:epoch 57 val auc: 0.7889712593409519
INFO:model.base_model:Epoch Step: 58 Train Loss: 0.5771361589431763 elapsed: 2.4978437423706055
INFO:model.base_model:cur_epoch:58, validation elapsed_time:0.24443793296813965
INFO:__main__:epoch 58 val auc: 0.7792087762164628
INFO:model.base_model:Epoch Step: 59 Train Loss: 0.5705540776252747 elapsed: 2.458335161209106

save_path ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0070.pth
Success to save to :  ./train_data/CategoricalAttentionAddScaledIntensityModel_dmodel64_nhead8_lr1e-05_dropout0.1_labelsoomthing0.1_0070.pth


INFO:model.base_model:Epoch Step: 71 Train Loss: 0.5691900253295898 elapsed: 2.3764901161193848
INFO:model.base_model:cur_epoch:71, validation elapsed_time:0.24404478073120117
INFO:__main__:epoch 71 val auc: 0.7875710672928827
INFO:model.base_model:Epoch Step: 72 Train Loss: 0.571700394153595 elapsed: 2.442237138748169
INFO:model.base_model:cur_epoch:72, validation elapsed_time:0.25644397735595703
INFO:__main__:epoch 72 val auc: 0.7750246002167671
INFO:model.base_model:Epoch Step: 73 Train Loss: 0.5728328824043274 elapsed: 2.5236170291900635
INFO:model.base_model:cur_epoch:73, validation elapsed_time:0.24608373641967773
INFO:__main__:epoch 73 val auc: 0.7784204997052728
INFO:model.base_model:Epoch Step: 74 Train Loss: 0.5672560930252075 elapsed: 2.391442060470581
INFO:model.base_model:cur_epoch:74, validation elapsed_time:0.2382650375366211
INFO:__main__:epoch 74 val auc: 0.7980332899165257
INFO:model.base_model:Epoch Step: 75 Train Loss: 0.5694180130958557 elapsed: 2.4391772747039795


In [ ]:
## Check the best model's validation performance
display(val_log)
print("Best validation performance:", max(val_log.values(), key=lambda x : x['auc']))

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = best_model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

## Transformer encoder
일단 3층짜리로 한번 돌려보자

### Transformer encoder Add intensity 

In [28]:
from model.attention_model import CategoricalTransformerEncoderAddIntensity

In [103]:


len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)
d_model = 128
n_head = 8
lr = 3e-5
n_layer = 3
dropout = 0.1
smoothing = 0.05

encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layer)
model = CategoricalTransformerEncoderAddIntensity(
    transformer_encoder=transformer_encoder,
    dict_category=dict_category,
    len_seq=len_seq
)

model.eval()

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss()
optimizer = optim.RAdam(model.parameters(), lr=lr)
batch_size = 64
epochs = 300

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

config_name = f"dmodel{d_model}_nlyaer{n_layer}_nhead{n_head}_lr{lr}_dropout{dropout}_labelsoomthing{smoothing}"
train_log = dict()
val_log = dict()
loss_func = nn.BCELoss()
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}

for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalTransformerEncoderAddIntensity.train_on_epoch(epoch, train_loader, model, loss_func, optimizer, verbose=True)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = CategoricalTransformerEncoderAddIntensity.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result
    logger.info(f"epoch {epoch} val result: {val_result}")

    if not val_log:
        best_model = save_model(config_name, model.copy(), epoch)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = save_model(config_name, model.copy(), epoch)
    

len_seq: 22
d_model: 128 n_head: 8 len_seq: 22


INFO:model.base_model:Epoch Step: 0 Train Loss: 0.7875859141349792 elapsed: 26.866644144058228
INFO:model.base_model:cur_epoch:0, validation elapsed_time:1.830441951751709
INFO:__main__:epoch 0 val result: {'loss': tensor(1.1677), 'auc': 0.7734607632484646}


save_path ./train_data/CategoricalTransformerEncoderAddIntensity_dmodel128_nlyaer3_nhead8_lr3e-05_dropout0.1_labelsoomthing0.05_0000.pth
Success to save to :  ./train_data/CategoricalTransformerEncoderAddIntensity_dmodel128_nlyaer3_nhead8_lr3e-05_dropout0.1_labelsoomthing0.05_0000.pth


INFO:model.base_model:Epoch Step: 1 Train Loss: 0.8075587153434753 elapsed: 27.792806148529053
INFO:model.base_model:cur_epoch:1, validation elapsed_time:1.5495977401733398
INFO:__main__:epoch 1 val result: {'loss': tensor(0.6428), 'auc': 0.7725170418892966}
INFO:model.base_model:Epoch Step: 2 Train Loss: 0.8248575925827026 elapsed: 26.6713969707489
INFO:model.base_model:cur_epoch:2, validation elapsed_time:1.5309369564056396
INFO:__main__:epoch 2 val result: {'loss': tensor(1.0113), 'auc': 0.7713496605883137}
INFO:model.base_model:Epoch Step: 3 Train Loss: 0.7471477389335632 elapsed: 26.02061915397644
INFO:model.base_model:cur_epoch:3, validation elapsed_time:1.536780834197998
INFO:__main__:epoch 3 val result: {'loss': tensor(0.8957), 'auc': 0.7660980490958529}
INFO:model.base_model:Epoch Step: 4 Train Loss: 0.8954782485961914 elapsed: 27.229969263076782
INFO:model.base_model:cur_epoch:4, validation elapsed_time:1.663275957107544
INFO:__main__:epoch 4 val result: {'loss': tensor(0.807

save_path ./train_data/CategoricalTransformerEncoderAddIntensity_dmodel128_nlyaer3_nhead8_lr3e-05_dropout0.1_labelsoomthing0.05_0005.pth
Success to save to :  ./train_data/CategoricalTransformerEncoderAddIntensity_dmodel128_nlyaer3_nhead8_lr3e-05_dropout0.1_labelsoomthing0.05_0005.pth


...

INFO:model.base_model:Epoch Step: 165 Train Loss: 2.0995876789093018 elapsed: 28.30774712562561
INFO:model.base_model:cur_epoch:165, validation elapsed_time:1.5667309761047363
INFO:__main__:epoch 165 val result: {'loss': tensor(0.6519), 'auc': 0.44296683367876644}
INFO:model.base_model:Epoch Step: 166 Train Loss: 1.2626060247421265 elapsed: 28.439980030059814
INFO:model.base_model:cur_epoch:166, validation elapsed_time:1.5895297527313232
INFO:__main__:epoch 166 val result: {'loss': tensor(1.8887), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 167 Train Loss: 2.460540533065796 elapsed: 28.928887844085693
INFO:model.base_model:cur_epoch:167, validation elapsed_time:1.5863230228424072
INFO:__main__:epoch 167 val result: {'loss': tensor(2.4712), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 168 Train Loss: 2.4237141609191895 elapsed: 29.158828020095825
INFO:model.base_model:cur_epoch:168, validation elapsed_time:1.7625970840454102
INFO:__main__:epoch 168 val result: {'loss': tensor(1.8607)

INFO:__main__:epoch 196 val result: {'loss': tensor(1.4714), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 197 Train Loss: 0.9718377590179443 elapsed: 29.803275108337402
INFO:model.base_model:cur_epoch:197, validation elapsed_time:1.611030101776123
INFO:__main__:epoch 197 val result: {'loss': tensor(0.6375), 'auc': 0.5743101243558785}
INFO:model.base_model:Epoch Step: 198 Train Loss: 1.1929420232772827 elapsed: 29.796180963516235
INFO:model.base_model:cur_epoch:198, validation elapsed_time:1.613576889038086
INFO:__main__:epoch 198 val result: {'loss': tensor(0.7691), 'auc': 0.3766419159171721}
INFO:model.base_model:Epoch Step: 199 Train Loss: 1.0710599422454834 elapsed: 28.297624826431274
INFO:model.base_model:cur_epoch:199, validation elapsed_time:1.6397287845611572
INFO:__main__:epoch 199 val result: {'loss': tensor(0.7975), 'auc': 0.4740245479264513}
INFO:model.base_model:Epoch Step: 200 Train Loss: 1.0783032178878784 elapsed: 28.73577904701233
INFO:model.base_model:cur_epoch:200, v

INFO:model.base_model:cur_epoch:228, validation elapsed_time:1.4551191329956055
INFO:__main__:epoch 228 val result: {'loss': tensor(0.5804), 'auc': 0.6469789270027191}
INFO:model.base_model:Epoch Step: 229 Train Loss: 1.2202452421188354 elapsed: 27.83061385154724
INFO:model.base_model:cur_epoch:229, validation elapsed_time:1.5702269077301025
INFO:__main__:epoch 229 val result: {'loss': tensor(1.3344), 'auc': 0.642854766024605}
INFO:model.base_model:Epoch Step: 230 Train Loss: 1.391523003578186 elapsed: 28.524535655975342
INFO:model.base_model:cur_epoch:230, validation elapsed_time:1.4695489406585693
INFO:__main__:epoch 230 val result: {'loss': tensor(2.1508), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 231 Train Loss: 1.2251291275024414 elapsed: 29.660919189453125
INFO:model.base_model:cur_epoch:231, validation elapsed_time:1.6023170948028564
INFO:__main__:epoch 231 val result: {'loss': tensor(0.8929), 'auc': 0.6463769941624994}
INFO:model.base_model:Epoch Step: 232 Train Loss: 1.383

INFO:model.base_model:cur_epoch:260, validation elapsed_time:1.8433139324188232
INFO:__main__:epoch 260 val result: {'loss': tensor(2.6099), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 261 Train Loss: 1.7942224740982056 elapsed: 30.603678941726685
INFO:model.base_model:cur_epoch:261, validation elapsed_time:1.9159767627716064
INFO:__main__:epoch 261 val result: {'loss': tensor(1.1488), 'auc': 0.5729880112566789}
INFO:model.base_model:Epoch Step: 262 Train Loss: 1.5869709253311157 elapsed: 29.020439863204956
INFO:model.base_model:cur_epoch:262, validation elapsed_time:1.7237792015075684
INFO:__main__:epoch 262 val result: {'loss': tensor(1.5079), 'auc': 0.5201511427810842}
INFO:model.base_model:Epoch Step: 263 Train Loss: 1.5372052192687988 elapsed: 28.951658010482788
INFO:model.base_model:cur_epoch:263, validation elapsed_time:1.7547740936279297
INFO:__main__:epoch 263 val result: {'loss': tensor(1.2369), 'auc': 0.5046735658192466}
INFO:model.base_model:Epoch Step: 264 Train Loss: 1.

INFO:model.base_model:Epoch Step: 292 Train Loss: 2.113201141357422 elapsed: 29.342538118362427
INFO:model.base_model:cur_epoch:292, validation elapsed_time:1.4722506999969482
INFO:__main__:epoch 292 val result: {'loss': tensor(2.2976), 'auc': 0.55079516932555}
INFO:model.base_model:Epoch Step: 293 Train Loss: 1.9975718259811401 elapsed: 29.52054190635681
INFO:model.base_model:cur_epoch:293, validation elapsed_time:1.4145519733428955
INFO:__main__:epoch 293 val result: {'loss': tensor(1.2960), 'auc': 0.6086074613527029}
INFO:model.base_model:Epoch Step: 294 Train Loss: 1.2456424236297607 elapsed: 29.957987070083618
INFO:model.base_model:cur_epoch:294, validation elapsed_time:1.4681730270385742
INFO:__main__:epoch 294 val result: {'loss': tensor(1.8186), 'auc': 0.6280980823715085}
INFO:model.base_model:Epoch Step: 295 Train Loss: 1.9296456575393677 elapsed: 30.337087154388428
INFO:model.base_model:cur_epoch:295, validation elapsed_time:1.4360909461975098
INFO:__main__:epoch 295 val resu

In [104]:
## Check the best model's validation performance
display(val_log)
print("Best validation performance:", max(val_log.values(), key=lambda x : x['auc']))

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = best_model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

{'epoch_val_0': {'loss': tensor(1.1677), 'auc': 0.7734607632484646},
 'epoch_val_1': {'loss': tensor(0.6428), 'auc': 0.7725170418892966},
 'epoch_val_2': {'loss': tensor(1.0113), 'auc': 0.7713496605883137},
 'epoch_val_3': {'loss': tensor(0.8957), 'auc': 0.7660980490958529},
 'epoch_val_4': {'loss': tensor(0.8073), 'auc': 0.7721518415698503},
 'epoch_val_5': {'loss': tensor(0.5743), 'auc': 0.8004809520640414},
 'epoch_val_6': {'loss': tensor(0.7762), 'auc': 0.713165750793862},
 'epoch_val_7': {'loss': tensor(1.1141), 'auc': 0.7157687389477286},
 'epoch_val_8': {'loss': tensor(1.7042), 'auc': 0.6620353292388432},
 'epoch_val_9': {'loss': tensor(1.0527), 'auc': 0.5503211100758685},
 'epoch_val_10': {'loss': tensor(1.9007), 'auc': 0.5389907493677626},
 'epoch_val_11': {'loss': tensor(1.0607), 'auc': 0.5223087125173509},
 'epoch_val_12': {'loss': tensor(3.1199), 'auc': 0.5},
 'epoch_val_13': {'loss': tensor(1.2113), 'auc': 0.3912596737084292},
 'epoch_val_14': {'loss': tensor(1.2838), 'auc

Best validation performance: {'loss': tensor(0.5743), 'auc': 0.8004809520640414}


INFO:model.base_model:cur_epoch:299, validation elapsed_time:1.44907808303833


{'loss': tensor(0.5888), 'auc': 0.8018167793433162}

### Transformer encoder Add Scaled intensity 

In [107]:
from model.attention_model import CategoricalTransformerEncoderAddScaledIntensityModel

In [108]:


len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)
d_model = 128
n_head = 8
lr = 1e-5
n_layer = 3
dropout = 0.1
smoothing = 0.05

encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layer)
model = CategoricalTransformerEncoderAddScaledIntensityModel(
    transformer_encoder=transformer_encoder,
    dict_category=dict_category,
    len_seq=len_seq,
    dropout=dropout
)

model.eval()

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss()
optimizer = optim.RAdam(model.parameters(), lr=lr)
batch_size = 64
epochs = 300

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

config_name = f"dmodel{d_model}_nlyaer{n_layer}_nhead{n_head}_lr{lr}_dropout{dropout}_labelsoomthing{smoothing}"
train_log = dict()
val_log = dict()
loss_func = nn.BCELoss()
val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}

for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalTransformerEncoderAddScaledIntensityModel.train_on_epoch(epoch, train_loader, model, loss_func, optimizer, verbose=True)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = CategoricalTransformerEncoderAddScaledIntensityModel.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result
    logger.info(f"epoch {epoch} val result: {val_result}")

    if not val_log:
        best_model = save_model(config_name, model.copy(), epoch)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = save_model(config_name, model.copy(), epoch)
    

len_seq: 22
d_model: 128 n_head: 8 len_seq: 22


INFO:model.base_model:Epoch Step: 0 Train Loss: 0.5557761192321777 elapsed: 26.315746784210205
INFO:model.base_model:cur_epoch:0, validation elapsed_time:1.8464078903198242
INFO:__main__:epoch 0 val result: {'loss': tensor(0.5033), 'auc': 0.769251630507121}


save_path ./train_data/CategoricalTransformerEncoderAddScaledIntensityModel_dmodel128_nlyaer3_nhead8_lr1e-05_dropout0.1_labelsoomthing0.05_0000.pth
Success to save to :  ./train_data/CategoricalTransformerEncoderAddScaledIntensityModel_dmodel128_nlyaer3_nhead8_lr1e-05_dropout0.1_labelsoomthing0.05_0000.pth


INFO:model.base_model:Epoch Step: 1 Train Loss: 0.6357571482658386 elapsed: 29.026705026626587
INFO:model.base_model:cur_epoch:1, validation elapsed_time:1.6667468547821045
INFO:__main__:epoch 1 val result: {'loss': tensor(0.6758), 'auc': 0.7759205472419235}


save_path ./train_data/CategoricalTransformerEncoderAddScaledIntensityModel_dmodel128_nlyaer3_nhead8_lr1e-05_dropout0.1_labelsoomthing0.05_0001.pth
Success to save to :  ./train_data/CategoricalTransformerEncoderAddScaledIntensityModel_dmodel128_nlyaer3_nhead8_lr1e-05_dropout0.1_labelsoomthing0.05_0001.pth


...

INFO:model.base_model:cur_epoch:174, validation elapsed_time:1.85300612449646
INFO:__main__:epoch 174 val result: {'loss': tensor(0.5418), 'auc': 0.7804879874883535}
INFO:model.base_model:Epoch Step: 175 Train Loss: 0.6254956126213074 elapsed: 31.890658140182495
INFO:model.base_model:cur_epoch:175, validation elapsed_time:2.0510268211364746
INFO:__main__:epoch 175 val result: {'loss': tensor(0.6572), 'auc': 0.7804417580954917}
INFO:model.base_model:Epoch Step: 176 Train Loss: 0.6732369661331177 elapsed: 32.68580198287964
INFO:model.base_model:cur_epoch:176, validation elapsed_time:2.0460100173950195
INFO:__main__:epoch 176 val result: {'loss': tensor(0.6397), 'auc': 0.7781792512026773}
INFO:model.base_model:Epoch Step: 177 Train Loss: 0.6011834144592285 elapsed: 30.17319130897522
INFO:model.base_model:cur_epoch:177, validation elapsed_time:1.7783088684082031
INFO:__main__:epoch 177 val result: {'loss': tensor(0.5161), 'auc': 0.7771185896826454}
INFO:model.base_model:Epoch Step: 178 Tra

INFO:__main__:epoch 205 val result: {'loss': tensor(0.5194), 'auc': 0.7738200214865661}
INFO:model.base_model:Epoch Step: 206 Train Loss: 0.5734808444976807 elapsed: 30.54332208633423
INFO:model.base_model:cur_epoch:206, validation elapsed_time:1.8959429264068604
INFO:__main__:epoch 206 val result: {'loss': tensor(0.5884), 'auc': 0.7734946331121295}
INFO:model.base_model:Epoch Step: 207 Train Loss: 0.6066810488700867 elapsed: 32.15801024436951
INFO:model.base_model:cur_epoch:207, validation elapsed_time:1.8601939678192139
INFO:__main__:epoch 207 val result: {'loss': tensor(0.5828), 'auc': 0.7733155387803996}
INFO:model.base_model:Epoch Step: 208 Train Loss: 0.5650143027305603 elapsed: 31.59193205833435
INFO:model.base_model:cur_epoch:208, validation elapsed_time:1.8956921100616455
INFO:__main__:epoch 208 val result: {'loss': tensor(0.5080), 'auc': 0.7734333108326519}
INFO:model.base_model:Epoch Step: 209 Train Loss: 0.5261702537536621 elapsed: 31.639616012573242
INFO:model.base_model:c

INFO:model.base_model:Epoch Step: 237 Train Loss: 0.5255749821662903 elapsed: 34.21296691894531
INFO:model.base_model:cur_epoch:237, validation elapsed_time:2.1478118896484375
INFO:__main__:epoch 237 val result: {'loss': tensor(0.4888), 'auc': 0.7571893242189729}
INFO:model.base_model:Epoch Step: 238 Train Loss: 0.5267849564552307 elapsed: 33.763335943222046
INFO:model.base_model:cur_epoch:238, validation elapsed_time:2.0485141277313232
INFO:__main__:epoch 238 val result: {'loss': tensor(0.5128), 'auc': 0.751484688444791}
INFO:model.base_model:Epoch Step: 239 Train Loss: 0.5444077849388123 elapsed: 33.694921255111694
INFO:model.base_model:cur_epoch:239, validation elapsed_time:2.0942273139953613
INFO:__main__:epoch 239 val result: {'loss': tensor(0.5357), 'auc': 0.7490105246144777}
INFO:model.base_model:Epoch Step: 240 Train Loss: 0.548441469669342 elapsed: 33.00613474845886
INFO:model.base_model:cur_epoch:240, validation elapsed_time:2.0335962772369385
INFO:__main__:epoch 240 val resu

INFO:model.base_model:cur_epoch:268, validation elapsed_time:1.8487038612365723
INFO:__main__:epoch 268 val result: {'loss': tensor(0.4948), 'auc': 0.735172962103782}
INFO:model.base_model:Epoch Step: 269 Train Loss: 0.5142788290977478 elapsed: 30.586429834365845
INFO:model.base_model:cur_epoch:269, validation elapsed_time:2.0944929122924805
INFO:__main__:epoch 269 val result: {'loss': tensor(0.4885), 'auc': 0.7389465402825579}
INFO:model.base_model:Epoch Step: 270 Train Loss: 0.5138311982154846 elapsed: 32.62400484085083
INFO:model.base_model:cur_epoch:270, validation elapsed_time:1.7983160018920898
INFO:__main__:epoch 270 val result: {'loss': tensor(0.4879), 'auc': 0.7274603782015934}
INFO:model.base_model:Epoch Step: 271 Train Loss: 0.5183818340301514 elapsed: 32.26038718223572
INFO:model.base_model:cur_epoch:271, validation elapsed_time:1.840212106704712
INFO:__main__:epoch 271 val result: {'loss': tensor(0.4890), 'auc': 0.7248562016314579}
INFO:model.base_model:Epoch Step: 272 Tra

INFO:__main__:epoch 299 val result: {'loss': tensor(0.5042), 'auc': 0.7208025137380921}


In [109]:
## Check the best model's validation performance
display(val_log)
print("Best validation performance:", max(val_log.values(), key=lambda x : x['auc']))

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = best_model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

{'epoch_val_0': {'loss': tensor(0.5033), 'auc': 0.769251630507121},
 'epoch_val_1': {'loss': tensor(0.6758), 'auc': 0.7759205472419235},
 'epoch_val_2': {'loss': tensor(0.7071), 'auc': 0.7824426945675116},
 'epoch_val_3': {'loss': tensor(0.5308), 'auc': 0.7816634500199654},
 'epoch_val_4': {'loss': tensor(0.5053), 'auc': 0.7800107195147458},
 'epoch_val_5': {'loss': tensor(0.4861), 'auc': 0.7802009849594038},
 'epoch_val_6': {'loss': tensor(0.5418), 'auc': 0.7793757487022496},
 'epoch_val_7': {'loss': tensor(0.6367), 'auc': 0.7794261375520526},
 'epoch_val_8': {'loss': tensor(0.6908), 'auc': 0.7783395685573576},
 'epoch_val_9': {'loss': tensor(0.8595), 'auc': 0.7789643189899412},
 'epoch_val_10': {'loss': tensor(0.7814), 'auc': 0.7765561122625545},
 'epoch_val_11': {'loss': tensor(0.7048), 'auc': 0.7777728128386987},
 'epoch_val_12': {'loss': tensor(0.5302), 'auc': 0.7804585147648837},
 'epoch_val_13': {'loss': tensor(0.5443), 'auc': 0.7832850440189387},
 'epoch_val_14': {'loss': tenso

Best validation performance: {'loss': tensor(0.5145), 'auc': 0.8018937413245611}


INFO:model.base_model:cur_epoch:299, validation elapsed_time:1.6498308181762695


{'loss': tensor(0.5213), 'auc': 0.8030242001106662}